In [17]:
import pandas as pd

# import the data
Shock = pd.read_csv('./Dataset_IPshocks/shocks_20250514_121541.csv')

Wind_Shock = Shock[Shock['Spacecraft'] == 'Wind']

Wind_helios = pd.read_csv('./Dataset_ICMECAT/helio4cast_icmecat.csv')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def GSE_sc_plot(plot_df, plot_df2, plot_title):
    # === Prepare plot_df (Shock events) ===
    event_time_dt_1 = pd.DatetimeIndex(plot_df['Time'])
    event_time_numeric_1 = (event_time_dt_1 - pd.Timestamp("1970-01-01")) / pd.Timedelta(days=1)
    event_dates_str_1 = event_time_dt_1.strftime('%Y-%m-%d')

    df_Shock = pd.DataFrame({
        'x': plot_df['SC_X'],
        'y': plot_df['SC_Y'],
        'z': plot_df['SC_Z'],
        'Event Time Numeric': event_time_numeric_1,
        'Event Time Label': event_dates_str_1,
        'Event Time': event_time_dt_1
    })

    # === Prepare plot_df2 (ICME events) ===
    event_time_dt_2 = pd.DatetimeIndex(plot_df2['sc_time'])
    event_time_numeric_2 = (event_time_dt_2 - pd.Timestamp("1970-01-01")) / pd.Timedelta(days=1)
    event_dates_str_2 = event_time_dt_2.strftime('%Y-%m-%d')

    df_helios = pd.DataFrame({
        'x': plot_df2['mo_sc_GSE_x'],
        'y': plot_df2['mo_sc_GSE_y'],
        'z': plot_df2['mo_sc_GSE_z'],
        'Event Time Numeric': event_time_numeric_2,
        'Event Time Label': event_dates_str_2,
        'Event Time': event_time_dt_2
    })

    # === Start with an empty figure ===
    fig = go.Figure()

    # === Add Shock scatter (black) ===
    fig.add_trace(go.Scatter3d(
        x=df_Shock['x'], y=df_Shock['y'], z=df_Shock['z'],
        mode='markers',
        marker=dict(size=3, color='black', opacity=0.4),
        name='Shock Catalogue',
        text=df_Shock['Event Time Label'],
        hoverinfo='text'
    ))

    # === Add ICME scatter (green) ===
    fig.add_trace(go.Scatter3d(
        x=df_helios['x'], y=df_helios['y'], z=df_helios['z'],
        mode='markers',
        marker=dict(size=3, color='green', opacity=0.6),
        name='helio4cast Catalogue', 
        text=df_helios['Event Time Label'],
        hoverinfo='text'
    ))

    # === Earth marker (blue) ===
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers+text',
        marker=dict(size=10, color='blue', line=dict(color='black', width=2)),
        text=['Earth'],
        textposition='top center',
        name='Earth',
        hoverinfo='text'
    ))

    # === L1 marker (red) ===
    fig.add_trace(go.Scatter3d(
        x=[238], y=[0], z=[0],
        mode='markers+text',
        marker=dict(size=5, color='red', line=dict(color='black', width=1)),
        text=['L1'],
        textposition='top center',
        name='L1',
        hoverinfo='text'
    ))

    # === Layout customization ===
    all_x = pd.concat([df_Shock['x'], df_helios['x']])
    all_y = pd.concat([df_Shock['y'], df_helios['y']])
    all_z = pd.concat([df_Shock['z'], df_helios['z']])

    fig.update_layout(
        title=plot_title,
        scene=dict(
            xaxis=dict(title='X [Earth radii]', range=[np.min(all_x), np.max(all_x)]),
            yaxis=dict(title='Y [Earth radii]', range=[np.min(all_y), np.max(all_y)]),
            zaxis=dict(title='Z [Earth radii]', range=[np.min(all_z), np.max(all_z)])
        ),
        legend=dict(x=0.8, y=0.9),
        margin=dict(l=0, r=0, b=0, t=50),
        width=800,
        height=600
    )

    fig.show()


In [19]:
GSE_sc_plot(Wind_Shock, Wind_helios, 'Wind Spacecraft in GSE Coordinates')